In [27]:
%pip install pandas
%pip install seaborn
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib as mpl
from typing import Optional
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score
from sklearn.metrics import classification_report, log_loss
import warnings
warnings.filterwarnings('ignore')

In [29]:
data = pd.read_csv('train.csv', low_memory=False, index_col='id')
data.head()

,forecasting period,net_profit_over_total_assets,total_liabilities_over_total_assets,working_capital_over_total_assets,current_assets_over_short_term_liabilities,cash_plus_short_term_securities_plus_receivables_minus_short_term_liabilities_over_operating_expenses_minus_depreciation_times_365,retained_earnings_over_total_assets,EBIT_over_total_assets,book_value_of_equity_over_total_liabilities,sales_over_total_assets,...,sales_minus_cost_of_products_sold_over_sales,current_assets_minus_inventory_minus_short_term_liabilities_over_sales_minus_gross_profit_minus_depreciation,total_costs_overtotal_sales,long_term_liabilities_over_equity,sales_over_inventory,sales_over_receivables,short_term_liabilities_times_365_over_sales,sales_over_short_term_liabilities,sales_over_fixed_assets,class
id,,,,,,,,,,,,,,,,,,,,,
0,4,0.14193928185699228,0.4502922641856586,0.021421961692346708,1.0529865561090996,-28.529083790353447,0.1419335154965603,0.17262900339766987,0.9903370061983934,1.0873559014030911,...,0.07957187794085908,0.3182855594575753,0.9276869666991867,0.10681814516207373,12.941797286204972,10.044584894139062,64.71369972433212,5.6268942646021545,3.9370641007164138,1
1,4,0.14756047866725389,0.23483997906033768,0.7083348449638224,4.016018367133703,119.44561073601812,0.4570896245318222,0.18606967096118351,3.2064449649495983,1.0934040907472207,...,0.09521770610492689,0.19594346640901797,0.9070561344356298,-2.926481932870128e-05,233.30168832258815,2.7474158180607167,36.967434520776315,9.813419478357071,40.56773917968368,0
2,1,-0.024542174376049268,0.31154432475503113,0.06783064757318172,1.2552471026972796,-15.01992268530473,-0.6570027776432179,-0.024536711275432783,2.209000357935884,0.8255784100560334,...,0.10903452563942133,-0.03563407515539158,0.9298251966432225,0.055331375472079294,8.589990830766858,5.164737918343701,117.92403966526238,3.098272636244345,1.2397105028735345,0
3,2,0.15931717015006794,0.7770168864172592,0.20743084665674563,1.2673100488694038,-4.892917756326703,-5.19193041774161e-06,0.15932116360799606,0.2881773632418629,2.2835319735778707,...,0.10297114004320836,0.7145277173151922,0.9378954347737054,-7.675610389379904e-05,9.92822118614928,5.91278624799987,123.7526101491789,2.9460083818399756,130.2083212255033,0
4,4,0.004403580663846555,0.5774189744272421,-0.10443457743771087,0.7862247169819696,-116.9116175491663,0.05656094534082444,0.004248643976367127,0.6921661681503327,1.013604581322834,...,0.016446056830358928,0.011019085112224006,0.9794252272096882,0.22133497785774525,5.099540696974632,6.890130245899504,166.7619795766402,2.189705498789674,1.7403533821469037,0


In [30]:
features = ['current_assets_over_short_term_liabilities', 'EBIT_over_total_assets', 'net_profit_over_total_assets', 'short_term_liabilities_over_total_assets', 'working_capital_over_total_assets', 'class']

In [31]:
datasets = data[features]
datasets.shape

(25121, 6)

In [32]:
for col in features:
    datasets.drop(datasets.loc[datasets[col] == '?'].index, inplace=True)
    # print(datasets.loc[datasets[col] == '?'].index)

In [33]:
datasets.shape

(25042, 6)

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
y = datasets['class'].values
y[y == 0] = -1
features.pop()
X = datasets[features].values

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=102)
print('Training set:', X_train.shape, y_train.shape)
print('Val set:', X_val.shape, y_val.shape)

Training set: (20033, 5) (20033,)
Val set: (5009, 5) (5009,)


In [37]:
def check(y):
    assert set(y) == {-1,1}
    return y

In [38]:
def init_model(iters, X):
    n = X.shape[0]
    sample_weights = np.zeros((iters, n))
    stumps = np.zeros(iters, dtype= object)
    stump_weights = np.zeros(iters)
    errors = np.zeros(iters)
    return stumps, stump_weights, sample_weights, errors

In [39]:
def AdaBoostClf(X, y, iters= 10):
    n = X.shape[0]
    # Check y
    y = check(y)
    # Initialize
    stumps, stump_weights, sample_weights, errors = init_model(iters= iters, X= X)

    # First weight = 1/n
    sample_weights[0] = np.ones(shape= n) / n

    for i in range(iters):
        # Fit for stump: weak learner
        current_sew = sample_weights[i]
        stump = DecisionTreeClassifier(max_depth= 1, max_leaf_nodes= 2)
        stump = stump.fit(X, y, sample_weight= current_sew)

        # Calculate error
        stump_pred = stump.predict(X)
        error = current_sew[stump_pred != y].sum()
        stump_weight = np.log((1 - error) / error) / 2

        # New sample weight
        new_sew = current_sew * np.exp(-1 * stump_weight * y * stump_pred)

        # Renormalize weights
        new_sew = new_sew / new_sew.sum()

        # If not last iter, update sample weights for i+1
        if (i + 1) < iters:
            sample_weights[i+1] = new_sew

        # Save result
        errors[i] = error
        stumps[i] = stump
        stump_weights[i] = stump_weight

    return stumps, stump_weights, sample_weights


In [40]:
def predict(X, stumps, stump_weights):
    stump_preds = np.array([stump.predict(X) for stump in stumps])
    return np.sign(np.dot(stump_weights, stump_preds))

In [41]:
# Training
stumps1, stump_weights1, sample_weights1 = AdaBoostClf(X= X_train, y= y_train.reshape(-1), iters= 10)
predt = predict(X_val, stumps1, stump_weights1)


In [47]:
# Show metrics
print("Accuracy score: %f" % accuracy_score(y_val, predt))
print("Confusion Matrix:")
print(confusion_matrix(y_val, predt))
print(classification_report(y_val, predt))
print('Log loss:', log_loss(y_val, predt)/len(y_val))

Accuracy score: 0.954682
Confusion Matrix:
[[4782    0]
 [ 227    0]]
              precision    recall  f1-score   support

          -1       0.95      1.00      0.98      4782
           1       0.00      0.00      0.00       227

    accuracy                           0.95      5009
   macro avg       0.48      0.50      0.49      5009
weighted avg       0.91      0.95      0.93      5009

Log loss: 0.00031248612715298644
